In [93]:
from flask import Flask, jsonify, request
from pymongo import MongoClient
from pprint import pprint

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello!'

In [94]:
client = MongoClient('mongodb://localhost:27017/')
db = client['Marvel-Vs-DC-Comic']
db2 = client['Marvel-Vs-DC-Movie']
comics_collection = db['Marvel-Vs-DC-Comic'] 
film_collection = db2['Marvel-Vs-DC-Movie']

In [95]:
@app.route('/api/dc_comics', methods=['GET'])
def get_dc_comics():
    dc_comics = list(comics_collection.find({"universe": "DC"}))

    return jsonify(dc_comics)

In [86]:
query = {'Universe':'Marvel'}
results = list(comics_collection.find(query))
print(len(results))

14665


In [87]:
query = {'Universe':'DC'}
results = list(comics_collection.find(query))
print(len(results))

6479


In [88]:
# How does a characters' identity influence their alignment(good, bad, neutral)?
@app.route('/api/character_identity_influence', methods=['GET'])
def character_identity_influence():
    identity_alignment_mapping = {
        "Secret": {"alignment": {"Good": 0, "Bad": 0, "Neutral": 0}},
        "Public": {"alignment": {"Good": 0, "Bad": 0, "Neutral": 0}},
        "Non-dual": {"alignment": {"Good": 0, "Bad": 0, "Neutral": 0}}
    }

    for identity, alignment_data in identity_alignment_mapping.items():
        alignment_count = alignment_data["alignment"]
        
        query = {"Identity": identity}
        characters = list(comics_collection.find(query))

        for character in characters:
            alignment = character["Alignment"]
            alignment_count[alignment] += 1

    return jsonify(identity_alignment_mapping)
if __name__ == '__main__':
    app.run(port=5000)
#http://127.0.0.1:5000/api/character_identity_influence

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Sep/2023 20:03:50] "GET /api/character_identity_influence HTTP/1.1" 200 -


In [92]:
# How does the number of good vs bad characters changes over x period of years in both DC and Marvel?
@app.route('/api/good_bad_character_changes', methods=['GET'])
def get_good_bad_character_changes():
    start_year = request.args.get('start_year', type=int)
    end_year = request.args.get('end_year', type=int)

    dc_query = {
        "Universe": "DC",
        "First_appeared_year": {"$gte": start_year, "$lte": end_year},
        "Alignment": {"$in": ["Good", "Bad"]}
    }

    marvel_query = {
        "Universe": "Marvel",
        "First_appeared_year": {"$gte": start_year, "$lte": end_year},
        "Alignment": {"$in": ["Good", "Bad"]}
    }

    dc_pipeline = [
        {"$match": dc_query},
        {"$group": {"_id": {"Year": "$First_appeared_year", "Alignment": "$Alignment"}, "count": {"$sum": 1}}}
    ]

    dc_results = list(comics_collection.aggregate(dc_pipeline))

    marvel_pipeline = [
        {"$match": marvel_query},
        {"$group": {"_id": {"Year": "$First_appeared_year", "Alignment": "$Alignment"}, "count": {"$sum": 1}}}
    ]

    marvel_results = list(comics_collection.aggregate(marvel_pipeline))

    response = {
        "DC_Characters": dc_results,
        "Marvel_Characters": marvel_results
    }

    return jsonify(response)
if __name__ == '__main__':
    app.run(port=5000)
# http://127.0.0.1:5000/api/character_count_over_time?start_year=2000&end_year=&universe=both&alignment=Good

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Sep/2023 20:05:13] "GET /api/character_count_over_time?start_year=2000&end_year=&universe=both&alignment=Good HTTP/1.1" 404 -


In [ ]:
# How does the number of appearances in DC/Marvel comics compare to the films released?

@app.route('/api/comics_vs_films', methods=['GET'])
def get_comics_vs_films():
    comics_count = comics_db['comics_collection'].count_documents({})
    
    films_count = films_db['film_collection'].count_documents({})

    response = {
        "comics_appearances_count": comics_count,
        "films_count": films_count
    }
    
    return jsonify(response)

if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2023-09-25 20:06:13,331] ERROR in app: Exception on /api/comics_vs_films [GET]
Traceback (most recent call last):
  File "C:\Users\hayde\anaconda3\lib\site-packages\flask\app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\hayde\anaconda3\lib\site-packages\flask\app.py", line 1486, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\hayde\anaconda3\lib\site-packages\flask\app.py", line 1484, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\hayde\anaconda3\lib\site-packages\flask\app.py", line 1469, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\hayde\AppData\Local\Temp\ipykernel_18016\2552267268.py", line 5, in get_comics_vs_films
    comics_count = comics_db['comics_collection'].count_documents({})
NameError: name 'comics_db' is not defined
127.0.0.1 - - [25/Sep/2023 

In [82]:
# Film Release : Is there a particular MPA rating that peforms better over another?

@app.route('/api/mpa_rating_performance', methods=['GET'])
def get_mpa_rating_performance():
    # Define the MPA ratings you want to analyze
    mpa_ratings = ["PG", "G", "PG-13", "R", "approved"]
    
    rating_performance = {}
    
    for rating in mpa_ratings:
        query = {"mpa_rating": rating}
        films = list(film_collection.find(query))
        
        if not films:
            continue
        
        total_imdb_rating = 0
        total_tomato_meter = 0
        total_tomato_review = 0
        total_tom_aud_score = 0
        film_count = 0
        
        for film in films:
            total_imdb_rating += film.get("imdb_rating", 0)
            total_tomato_meter += film.get("tomato_meter", 0)
            total_tomato_review += film.get("tomato_review", 0)
            total_tom_aud_score += film.get("tom_aud_score", 0)
            film_count += 1
        
        if film_count > 0:
            average_imdb_rating = total_imdb_rating / film_count
            average_tomato_meter = total_tomato_meter / film_count
            average_tomato_review = total_tomato_review / film_count
            average_tom_aud_score = total_tom_aud_score / film_count
            rating_performance[rating] = {
                "average_imdb_rating": average_imdb_rating,
                "average_tomato_meter": average_tomato_meter,
                "average_tomato_review": average_tomato_review,
                "average_tom_aud_score": average_tom_aud_score,
                "film_count": film_count
            }
    
    return jsonify(rating_performance)

if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Sep/2023 20:02:46] "GET /api/mpa_rating_performance HTTP/1.1" 200 -
